In [ ]:
# --- Colab Setup (run this cell first) ---
!pip install ultralytics tqdm transformers bitsandbytes accelerate jsonlines --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.5 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import json
import random
from ultralytics import YOLO
from tqdm import tqdm
from PIL import Image
import torch
import jsonlines
from transformers import AutoProcessor, AutoModelForVision2Seq, BitsAndBytesConfig


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
model_id = "llava-hf/llava-1.5-7b-hf"
processor = AutoProcessor.from_pretrained(model_id)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="cuda"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [ ]:
# --- CONFIG ---
POPE_IMG_ROOT = '/content/drive/MyDrive/UCLA/cs 263/final_proj/val2014/'  # Images are in the same directory level or update to your Drive path
POPE_ANN_PATH = '/content/drive/MyDrive/UCLA/cs 263/final_proj/coco_pope_adversarial (1).json'  # Your annotation file
RESULTS_FILE = '/content/drive/MyDrive/UCLA/cs 263/final_proj/pope_yolo_llava_results.json'
NUM_EXAMPLES = 500  # Number of unique images to process

In [ ]:
# --- 1. Load POPE Annotation Data (grouped by image, random sample) ---
def load_pope_annotations_grouped(ann_path, img_root, num_examples=100):
    """
    Loads POPE annotations and groups them by image filename.
    Randomly samples up to num_examples unique images.
    Returns a list of dicts: {img_path, questions: [annotation_dicts]}
    """
    from collections import defaultdict
    images = []
    image_to_questions = defaultdict(list)
    with open(ann_path, 'r') as f:
        for line in f:
            item = json.loads(line)
            img_fn = item['image']
            img_path = os.path.join(img_root, img_fn)
            if os.path.exists(img_path):
                print('loading:', img_path)
                image_to_questions[img_path].append(item)
    all_img_paths = list(image_to_questions.keys())
    random.shuffle(all_img_paths)
    all_img_paths = all_img_paths[:num_examples]
    for img_path in all_img_paths:
        images.append({'img_path': img_path, 'questions': image_to_questions[img_path]})
    return images

In [ ]:
  images = load_pope_annotations_grouped(POPE_ANN_PATH, POPE_IMG_ROOT, NUM_EXAMPLES)
  print(f"Loaded {len(images)} unique images from POPE dataset.")

loading: /content/drive/MyDrive/UCLA/cs 263/final_proj/val2014/COCO_val2014_000000310196.jpg
loading: /content/drive/MyDrive/UCLA/cs 263/final_proj/val2014/COCO_val2014_000000310196.jpg
loading: /content/drive/MyDrive/UCLA/cs 263/final_proj/val2014/COCO_val2014_000000310196.jpg
loading: /content/drive/MyDrive/UCLA/cs 263/final_proj/val2014/COCO_val2014_000000310196.jpg
loading: /content/drive/MyDrive/UCLA/cs 263/final_proj/val2014/COCO_val2014_000000310196.jpg
loading: /content/drive/MyDrive/UCLA/cs 263/final_proj/val2014/COCO_val2014_000000210789.jpg
loading: /content/drive/MyDrive/UCLA/cs 263/final_proj/val2014/COCO_val2014_000000210789.jpg
loading: /content/drive/MyDrive/UCLA/cs 263/final_proj/val2014/COCO_val2014_000000210789.jpg
loading: /content/drive/MyDrive/UCLA/cs 263/final_proj/val2014/COCO_val2014_000000210789.jpg
loading: /content/drive/MyDrive/UCLA/cs 263/final_proj/val2014/COCO_val2014_000000210789.jpg
loading: /content/drive/MyDrive/UCLA/cs 263/final_proj/val2014/COCO_va

In [ ]:
print(images[0])

{'img_path': '/content/drive/MyDrive/UCLA/cs 263/final_proj/val2014/COCO_val2014_000000458325.jpg', 'questions': [{'question_id': 2467, 'image': 'COCO_val2014_000000458325.jpg', 'text': 'Is there a car in the image?', 'label': 'yes'}, {'question_id': 2468, 'image': 'COCO_val2014_000000458325.jpg', 'text': 'Is there a truck in the image?', 'label': 'no'}, {'question_id': 2469, 'image': 'COCO_val2014_000000458325.jpg', 'text': 'Is there a traffic light in the image?', 'label': 'yes'}, {'question_id': 2470, 'image': 'COCO_val2014_000000458325.jpg', 'text': 'Is there a bus in the image?', 'label': 'no'}, {'question_id': 2471, 'image': 'COCO_val2014_000000458325.jpg', 'text': 'Is there a person in the image?', 'label': 'yes'}, {'question_id': 2472, 'image': 'COCO_val2014_000000458325.jpg', 'text': 'Is there a dining table in the image?', 'label': 'no'}]}


In [ ]:
# --- 2. Run YOLO Object Detection ---
def run_yolo_on_images(images, yolo_model_path='yolov8x.pt'):
    yolo_model = YOLO(yolo_model_path)
    results_dict = {}
    for entry in tqdm(images, desc='Running YOLO'):
        img_path = entry['img_path']
        try:
            results = yolo_model(img_path)
            detected_objects = [yolo_model.model.names[int(cls)] for cls in results[0].boxes.cls]
        except Exception as e:
            print(f"YOLO failed on {img_path}: {e}")
            detected_objects = []
        results_dict[img_path] = detected_objects
    return results_dict

In [ ]:
  yolo_results = run_yolo_on_images(images)
  print("YOLO detection complete.")

100%|██████████| 131M/131M [00:01<00:00, 89.3MB/s]
Running YOLO:   0%|          | 0/500 [00:00<?, ?it/s]


image 1/1 /content/drive/MyDrive/UCLA/cs 263/final_proj/val2014/COCO_val2014_000000175506.jpg: 448x640 2 persons, 1 frisbee, 112.4ms
Speed: 10.3ms preprocess, 112.4ms inference, 330.8ms postprocess per image at shape (1, 3, 448, 640)


Running YOLO:   0%|          | 1/500 [00:04<40:27,  4.87s/it]


image 1/1 /content/drive/MyDrive/UCLA/cs 263/final_proj/val2014/COCO_val2014_000000196462.jpg: 480x640 1 person, 2 cups, 1 fork, 1 bowl, 1 dining table, 1 cell phone, 78.2ms
Speed: 1.8ms preprocess, 78.2ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)


Running YOLO:   0%|          | 2/500 [00:05<21:05,  2.54s/it]

...


image 1/1 /content/drive/MyDrive/UCLA/cs 263/final_proj/val2014/COCO_val2014_000000449432.jpg: 448x640 14 persons, 1 bus, 3 backpacks, 3 handbags, 1 suitcase, 72.5ms
Speed: 1.7ms preprocess, 72.5ms inference, 1.7ms postprocess per image at shape (1, 3, 448, 640)


Running YOLO: 100%|█████████▉| 499/500 [09:04<00:01,  1.10s/it]


image 1/1 /content/drive/MyDrive/UCLA/cs 263/final_proj/val2014/COCO_val2014_000000538236.jpg: 448x640 5 persons, 11 donuts, 71.6ms
Speed: 1.6ms preprocess, 71.6ms inference, 2.0ms postprocess per image at shape (1, 3, 448, 640)


Running YOLO: 100%|██████████| 500/500 [09:05<00:00,  1.09s/it]

YOLO detection complete.


In [ ]:
#yolo_results

In [ ]:
# --- 3. Construct Prompts for LLaVA ---
def construct_yolo_prompt(detected_objects, question):
    prompt = "<image>\n"
    prompt += f"USER: This is a list of detected objects: {', '.join(detected_objects)}.\n"
    prompt += f"Look at the image and answer the following question:\n- {question}\n"
    prompt += "\nASSISTANT:"
    return prompt
def construct_normal_prompt(question):
    prompt = "<image>\n"
    prompt += f"USER: Look at the image and answer the following question:\n- {question}\n"
    prompt += "\nASSISTANT:"
    return prompt

In [ ]:
def run_llava_on_images_yolo(images, yolo_results):
    results = {}
    for entry in tqdm(images, desc='Running LLaVA'):
        img_path = entry['img_path']
        detected_objects = yolo_results.get(img_path, [])
        image = Image.open(img_path).convert("RGB")
        per_question_results = []
        #print(img_path)
        #print('Detected objects:', detected_objects)
        for q in entry['questions']:
            question = q['text']
            prompt = construct_yolo_prompt(detected_objects, question)
            #print(question)
            try:
                inputs = processor(prompt, image, return_tensors="pt").to(model.device)
                with torch.no_grad():
                    output = model.generate(**inputs, max_new_tokens=128)
                description = processor.batch_decode(output, skip_special_tokens=True)[0]
                # split on ASSISTANT:
                if "ASSISTANT:" in description:
                    description = description.split("ASSISTANT:")[-1].strip()
                #print(description)
            except Exception as e:
                print(f"LLaVA failed on {img_path} for question '{question}': {e}")
                description = "[ERROR] LLaVA inference failed."
            per_question_results.append({
                'question': question,
                'llava_answer': description,
                'label': q.get('label', None)
            })
        results[img_path] = {
            'detected_objects': detected_objects,
            'questions': per_question_results
        }
        #print('-'*50)
    return results

In [ ]:
def run_llava_on_images_normal(images):
    results = {}
    for entry in tqdm(images, desc='Running LLaVA'):
        img_path = entry['img_path']
        image = Image.open(img_path).convert("RGB")
        per_question_results = []
        #print(img_path)
        for q in entry['questions']:
            question = q['text']
            prompt = construct_normal_prompt(question)
            #print(question)
            try:
                inputs = processor(prompt, image, return_tensors="pt").to(model.device)
                with torch.no_grad():
                    output = model.generate(**inputs, max_new_tokens=128)
                description = processor.batch_decode(output, skip_special_tokens=True)[0]
                # split on ASSISTANT:
                if "ASSISTANT:" in description:
                    description = description.split("ASSISTANT:")[-1].strip()
                #print(description)
            except Exception as e:
                print(f"LLaVA failed on {img_path} for question '{question}': {e}")
                description = "[ERROR] LLaVA inference failed."
            per_question_results.append({
                'question': question,
                'llava_answer': description,
                'label': q.get('label', None)
            })
        results[img_path] = {
            'questions': per_question_results
        }
        #print('-'*50)
    return results

In [ ]:
llava_results = run_llava_on_images_yolo(images, yolo_results)
with open('/content/drive/MyDrive/UCLA/cs 263/final_proj/pope_yolo_llava_results.json', 'w') as f:
    json.dump(llava_results, f, indent=2)

Running LLaVA: 100%|██████████| 500/500 [53:15<00:00,  6.39s/it]


In [ ]:
llava_normal_results = run_llava_on_images_normal(images)
with open('/content/drive/MyDrive/UCLA/cs 263/final_proj/pope_normal_llava_results.json', 'w') as f:
    json.dump(llava_normal_results, f, indent=2)


Running LLaVA: 100%|██████████| 500/500 [1:02:54<00:00,  7.55s/it]


In [ ]:
import json
import re

In [ ]:
def extract_yes_no_from_answer(answer):
    """
    Extracts a yes/no prediction from the LLaVA answer.
    Returns 'yes', 'no', or None if unclear.
    """
    answer = answer.lower()
    # Look for 'yes' or 'no' as a standalone word at the start or after punctuation
    if re.match(r'^(yes)[\\W_]*', answer):
        return 'yes'
    if re.match(r'^(no)[\\W_]*', answer):
        return 'no'
    # Fallback: look for yes/no anywhere
    if 'yes' in answer:
        return 'yes'
    if 'no' in answer:
        return 'no'
    return None

In [ ]:
def evaluate_results(results_path):
    with open(results_path, 'r') as f:
        results = json.load(f)
    total = 0
    correct = 0
    for img_path, entry in results.items():
        for q in entry['questions']:
            label = q.get('label', '').strip().lower()
            answer = q.get('llava_answer', '')
            pred = extract_yes_no_from_answer(answer)
            if label in ['yes', 'no'] and pred is not None:
                total += 1
                if pred == label:
                    correct += 1
    accuracy = correct / total if total > 0 else 0
    print(f"Accuracy: {accuracy:.3f} ({correct}/{total})")
    return accuracy


In [8]:
import json, re

In [6]:
def extract_yes_no_from_answer(answer):
    """
    Extracts a yes/no prediction from the LLaVA answer.
    Returns 'yes', 'no', or None if unclear.
    """
    answer = answer.lower()
    # Look for 'yes' or 'no' as a standalone word at the start or after punctuation
    if re.match(r'^(yes)[\\W_]*', answer):
        return 'yes'
    if re.match(r'^(no)[\\W_]*', answer):
        return 'no'
    # Fallback: look for yes/no anywhere
    if 'yes' in answer:
        return 'yes'
    if 'no' in answer:
        return 'no'
    return None

In [13]:
def evaluate_results(results_path):
    with open(results_path, 'r') as f:
        results = json.load(f)

    # Metrics counters
    metrics = {
        'total': 0,
        'correct': 0,
        'tp': 0,  # true positives (predicted yes, label yes)
        'tn': 0,  # true negatives (predicted no, label no)
        'fp': 0,  # false positives (predicted yes, label no)
        'fn': 0,  # false negatives (predicted no, label yes)
        'hallucinated': 0,
        'hallucination_total': 0
    }

    # Stopwords for hallucination filtering
    stopwords = {
        'the','a','an','is','are','was','were','it','this','that','and','or','of','to','in','on','at','for','with','as','by','from','yes','no','not','but','if','then','so','because','about','into','over','after','before','between','during','without','within','through','above','below','up','down','out','off','again','further','once'
    }

    for img_path, entry in results.items():
        detected_objects = set(obj.lower() for obj in entry.get('detected_objects', []))
        for q in entry['questions']:
            label = q.get('label', '').strip().lower()
            answer = q.get('llava_answer', '')
            pred = extract_yes_no_from_answer(answer)
            # Classification metrics
            if label in ['yes', 'no'] and pred is not None:
                metrics['total'] += 1
                if pred == label:
                    metrics['correct'] += 1
                if label == 'yes' and pred == 'yes':
                    metrics['tp'] += 1
                elif label == 'no' and pred == 'no':
                    metrics['tn'] += 1
                elif label == 'no' and pred == 'yes':
                    metrics['fp'] += 1
                elif label == 'yes' and pred == 'no':
                    metrics['fn'] += 1
            # Hallucination metrics
            if 'detected_objects' in entry:
                metrics['hallucination_total'] += 1
                words = set(re.findall(r'\b\w+\b', answer.lower()))
                content_words = words - stopwords
                if len(detected_objects) > 0 and any(w not in detected_objects for w in content_words):
                    metrics['hallucinated'] += 1

    # Calculate scores
    accuracy = metrics['correct'] / metrics['total'] if metrics['total'] > 0 else 0
    precision = metrics['tp'] / (metrics['tp'] + metrics['fp']) if (metrics['tp'] + metrics['fp']) > 0 else 0
    recall = metrics['tp'] / (metrics['tp'] + metrics['fn']) if (metrics['tp'] + metrics['fn']) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    hallucination_rate = metrics['hallucinated'] / metrics['hallucination_total'] if metrics['hallucination_total'] > 0 else 0
    print(metrics)

    # Print results
    #print(f"Accuracy: {accuracy:.3f} ({metrics['correct']}/{metrics['total']})")
    #print(f"Precision: {precision:.3f}")
    #print(f"Recall: {recall:.3f}")
    #print(f"F1 Score: {f1:.3f}")
    #print(f"Hallucination Rate: {hallucination_rate:.3f} ({metrics['hallucinated']}/{metrics['hallucination_total']})")
    return accuracy, precision, recall, f1, hallucination_rate

In [14]:
yolo_acc, yolo_prec, yolo_rec, yolo_f1, yolo_hall = evaluate_results('/content/drive/MyDrive/UCLA/cs 263/final_proj/pope_yolo_llava_results.json')
normal_acc, normal_prec, normal_rec, normal_f1, normal_hall = evaluate_results('/content/drive/MyDrive/UCLA/cs 263/final_proj/pope_normal_llava_results.json')

print(f'YOLO-grounded accuracy: {yolo_acc:.3f}')
print(f'YOLO-grounded precision: {yolo_prec:.3f}')
print(f'YOLO-grounded recall: {yolo_rec:.3f}')
print(f'YOLO-grounded F1: {yolo_f1:.3f}')
print(f'YOLO-grounded hallucination rate: {yolo_hall:.3f}')

print(f'Normal accuracy: {normal_acc:.3f}')
print(f'Normal precision: {normal_prec:.3f}')
print(f'Normal recall: {normal_rec:.3f}')
print(f'Normal F1: {normal_f1:.3f}')
print(f'Normal hallucination rate: {normal_hall:.3f}')

{'total': 2999, 'correct': 2655, 'tp': 1282, 'tn': 1373, 'fp': 128, 'fn': 216, 'hallucinated': 1182, 'hallucination_total': 2999}
{'total': 2999, 'correct': 2460, 'tp': 1238, 'tn': 1222, 'fp': 279, 'fn': 260, 'hallucinated': 0, 'hallucination_total': 0}
YOLO-grounded accuracy: 0.885
YOLO-grounded precision: 0.909
YOLO-grounded recall: 0.856
YOLO-grounded F1: 0.882
YOLO-grounded hallucination rate: 0.394
Normal accuracy: 0.820
Normal precision: 0.816
Normal recall: 0.826
Normal F1: 0.821
Normal hallucination rate: 0.000


In [9]:
yolo_acc = evaluate_results('/content/drive/MyDrive/UCLA/cs 263/final_proj/pope_yolo_llava_results.json')
normal_acc = evaluate_results('/content/drive/MyDrive/UCLA/cs 263/final_proj/pope_normal_llava_results.json')
print(f'YOLO-grounded accuracy: {yolo_acc:.3f}')
print(f'Normal accuracy: {normal_acc:.3f}')

Accuracy: 0.885 (2655/2999)
Precision: 0.909
Recall: 0.856
F1 Score: 0.882
Hallucination Rate: 0.394 (1182/2999)
Accuracy: 0.820 (2460/2999)
Precision: 0.816
Recall: 0.826
F1 Score: 0.821
Hallucination Rate: 0.000 (0/0)


TypeError: unsupported format string passed to tuple.__format__